In [1]:
# Libraries:
import pysindy as ps
import numpy as np
import matplotlib.pyplot as plt
import dill
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # include parent directory in the path
from data import SINDy_data
from data import data
from data import equations

sys.path.append("/home/mattg/D_CODE") # A questo punto è necessario per non doverlo chiamare i file che usano D_CODE.
from D_CODE.run_simulation_vi import run_HD as run_DCODE

from toolbox.auxiliary_functions import set_param_freq, intercept_library_fun

# Seed:
np.random.seed(999)

/home/mattg/miniforge3/envs/SimbSINDy_env/lib/python3.11/site-packages/pysindy/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound


In [2]:
z = np.load("latent_data_fair.npy") 
print(z.shape) # shape: (1600, 2)

z = z[np.newaxis, :, :]
print(z.shape) # shape: (1, 1600, 2)

(1600, 2)
(1, 1600, 2)


In [3]:
# rimuovo la dimensione batch
z0 = z[0]                 # shape: (1600, 2)

# split in due parti
z1 = z0[:800, :]          # shape: (800, 2)
z2 = z0[800:, :]          # shape: (800, 2)

# stack lungo una nuova dimensione
z = np.stack([z1, z2], axis=0)

print(z.shape)       # (2, 800, 2)

(2, 800, 2)


In [4]:
# Select ODE & settings:
ode_name = 'DoubleGyre' # help="name of the ode", type=str
ode_param = '1,1' # help="parameters of the ode (default: None)", type=str, default=None
x_id = 0 # help="ID of the equation to be learned", type=int, default=0
freq = 20 # help="sampling frequency", type=float, default=10
n_sample = 1 # help="number of trajectories", type=int, default=100
noise_ratio = 0 # help="noise level (default 0)", type=float, default=0.
seed = 100 # help="random seed", type=int, default=0
n_seed = 1 # help="random seed", type=int, default=10

ode_param, freq = set_param_freq(ode_param, freq)
ode = equations.get_ode(ode_name, ode_param)
dt = 1 / freq
dim_x = 2
dim_k = 0

T0 = 0
T = (z.shape[1] - 1)/freq

latent_data = z.transpose(1, 0, 2)

In [5]:
# additional building blocks -> running D-CODE:
building_blocks_lambda, function_names = run_DCODE(ode_name=ode_name, ode_param=ode_param, x_id=0,
                                                   freq=freq, n_sample=n_sample, noise_ratio=noise_ratio, 
                                                   seed=seed, n_seed=n_seed, T0=T0, T=T, 
                                                   latent_data=latent_data)

Dataset shape:  (800, 2, 2)
Functions set:  {'neg': 1, 'mul': 1, 'add': 1, 'sub': 1, 'sin': 1, 'cos': 1}
 
    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    11.77      1.93995e+18        7          12.5743           82.102      2.19m
   1     9.49      1.17834e+11       14          6.84239          132.138      1.84m
   2     4.41      1.21751e+11       14          6.84239          132.138      1.07m
 
promising programs:
cos(mul(neg(4.487), add(3.401, X1)))
neg(cos(mul(X1, neg(4.487))))
sin(mul(4.465, X1))
sin(mul(4.465, X1))
sin(mul(4.465, X1))
sin(mul(4.465, X1))
sin(mul(4.465, X1))
sin(mul(4.465, X1))
sin(mul(4.465, X1))
cos(sub(mul(4.465, X1), neg(4.487)))
mul(2.765, cos(mul(5.314, X1)))
cos(mul(X1, neg(4.267)))
cos(mul(X1, neg(4.267)))
cos(mul(X1, neg(4.267)))
cos(mul(X1, neg(4.26

In [8]:
degree = 3
threshold = 0.08
penalty = 10

# generate data to run SINDy:
X_list, dX_list, param_list, feature_names = SINDy_data.SINDy_data_HD(ode_name=ode_name, ode_param=ode_param, 
                                                                      freq=freq, n_sample=n_sample, 
                                                                      noise_ratio=0, dim_x=dim_x, dim_k=dim_k,
                                                                      T0=T0, T=T, latent_data = latent_data)
print(np.shape(X_list), np.shape(dX_list), np.shape(param_list))
print(feature_names)

(2, 798, 2) (2, 798, 2) ()
['X0', 'X1']


In [9]:
print('Searching for the best building block:')

errors = []
n_features_vec = []
intercept_library = intercept_library_fun(dim_x+dim_k) # intercept library
polynomial_library = ps.PolynomialLibrary(degree=degree, include_bias=False) # polynomial library

for i in range(len(building_blocks_lambda)):

    # building the library:
    custom_library = ps.CustomLibrary(library_functions=[building_blocks_lambda[i]], function_names=[function_names[i]]) # custom library with building block
    generalized_library = ps.ConcatLibrary(libraries=[polynomial_library, custom_library])
    final_library = ps.ConcatLibrary([intercept_library, generalized_library]) # add the intercept

    # fitting the model:
    model = ps.SINDy(feature_names=feature_names, feature_library=final_library, optimizer=ps.STLSQ(threshold=threshold))
    model.fit(X_list, t=dt, multiple_trajectories=True, x_dot=dX_list)
    print('Model:')
    model.print()   

    # print('')
    # print('library:')
    # library_terms = final_library.get_feature_names(input_features=feature_names)
    # for term in library_terms:
    #     print(term)
    # print()   

    # evaluate the model:  
    coefficients = model.coefficients()
    model_complexity = np.count_nonzero(np.array(model.coefficients()))
    lasso_penalty = np.sum(np.abs(coefficients))
    if model_complexity < penalty and lasso_penalty < penalty: #filter too complex models (for sure not correct and likely to crash the code):
        _, mse = SINDy_data.evaluate_RMSE_HD(model=model, latent_data=X_list, freq=20, n_sample=2, T0=T0, T=T, dim_k=dim_k) # compute MSE      
        alpha = 0.01 # regularization parameter
        error = mse + alpha * lasso_penalty # final evaluation metric
        # print('error:', error)
    else:
        error = 1000
        #print('Too complex model')
    #print('')
    errors.append(error)
    n_features_vec.append(np.count_nonzero(np.array(model.coefficients())))

print("errors: ", errors)
if all(err == 1000 for err in errors):
    print('No model update, all smart-SINDy models are too complex')

else:
    # Final model
    min_error = min(errors)
    idxs = [i for i, e in enumerate(errors) if abs(e - min_error) < 0.01]
    n_features_vec_2 = [n_features_vec[i] for i in idxs]

    if len(idxs) > 1:
        # print('Multiple models with similar error, selecting the simplest one with the lowest error')
        # print('')
        min_features = min(n_features_vec_2) # find the min number of features among the candidates
        idxs_min_feat = [idxs[i] for i, nf in enumerate(n_features_vec_2) if nf == min_features] # select al the indexes with that number of features
        idx = idxs_min_feat[np.argmin([errors[i] for i in idxs_min_feat])] # among these, choose the one with less error
    else:
        idx = idxs[0]

    # building the library:
    custom_library = ps.CustomLibrary(library_functions=[building_blocks_lambda[idx]], function_names=[function_names[idx]])  # custom library with building block
    model = ps.SINDy(feature_names=feature_names, feature_library=custom_library, optimizer=ps.STLSQ(threshold=0.01))
    model.fit(X_list, t=dt, multiple_trajectories=True, x_dot=dX_list)
    building_block = custom_library.get_feature_names(input_features=feature_names) 
    generalized_library = ps.ConcatLibrary(libraries=[polynomial_library, custom_library])
    final_library = ps.ConcatLibrary([intercept_library, generalized_library]) # add the intercept

    # fitting the model:
    model = ps.SINDy(feature_names=feature_names, feature_library=final_library, optimizer=ps.STLSQ(threshold=threshold))
    model.fit(X_list, t=dt, multiple_trajectories=True, x_dot=dX_list)

    # best building block:
    print('')
    print('Best building block:')
    print(building_block)
    print('')

    # final model:
    print('Smart-SINDy model:')
    model.print()

Searching for the best building block:
Model:
(X0)' = 0.000
(X1)' = 1.000 1
Model:
(X0)' = 0.000
(X1)' = 1.000 1


/home/mattg/miniforge3/envs/SimbSINDy_env/lib/python3.11/site-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.08) and eliminated all coefficients
  warnings.warn(
/home/mattg/miniforge3/envs/SimbSINDy_env/lib/python3.11/site-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.08) and eliminated all coefficients
  warnings.warn(
/home/mattg/miniforge3/envs/SimbSINDy_env/lib/python3.11/site-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.08) and eliminated all coefficients
  warnings.warn(
/home/mattg/miniforge3/envs/SimbSINDy_env/lib/python3.11/site-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.08) and eliminated all coefficients
  warnings.warn(


Model:
(X0)' = 0.000
(X1)' = 1.000 1
Model:
(X0)' = 0.000
(X1)' = 1.000 1
Model:
(X0)' = 3.751 cos(4.998442071146355*X1)
(X1)' = 1.000 1
Model:
(X0)' = 0.000
(X1)' = 1.000 1
errors:  [0.22800444167774125, 0.22800444167774125, 0.22800444167774125, 0.22800444167774125, 0.0492116388498138, 0.22800444167774125]

Best building block:
['cos(4.998442071146355*X1)']

Smart-SINDy model:
(X0)' = 3.751 cos(4.998442071146355*X1)
(X1)' = 1.000 1


/home/mattg/miniforge3/envs/SimbSINDy_env/lib/python3.11/site-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.08) and eliminated all coefficients
  warnings.warn(
/home/mattg/miniforge3/envs/SimbSINDy_env/lib/python3.11/site-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.01) and eliminated all coefficients
  warnings.warn(
